# Brainstorm Proyecto 2 - Grafos

El primer paso es definir las entradas y salidas del problema

| Entrada / Salida | Nombre | Tipo | Descripción |
|-|-|-|-|
| E | C | **List** of **Celulas** | El conjunto de celulas donde encontramos su identificador, su posición, su tipo y sus péptidos asociados |
| E | d | **nat** | Distancia máxima entre células que se pueden enviar mensajes |
| s | c | **nat** | Identificador de la célula calculadora que se va a  |
| s | M | **nat** | Cantidad máxima de mensajes que se pueden enviar sin quitar células |
| s | m | **nat** | Cantidad máxima de mensajes que se pueden enviar quitando la célula calculadora indicada |

A partir del esquema mencionado en el enunciado, se tiene un problema  de flujo. Esto por lo siguiente:

1. La cantidad de mensajes recibidos y emitidos por las celulas calculadoras deben ser idénticos (Se de cumplir el principio de conservación).
2. Las celulas iniciadores reciben impulsos externos y envian mensajes a las celulas calculadoras (Son las fuentes de mensajes)
3. Las celulas ejecutadores reciben los mensajes de las celulas alculadores y ejecutan acciones en base a eso (Son los destinos de los mensajes).
4. La cantidad máxima de mensajes que se pueden enviar entre celulas esta definido por la cantidad de péptidos compartidos (Existe una capacidad máxima de mensajes que se pueden enviar entre celulas calculadoras)

Dado esto, podemos definir que la siguiente idea de algoritmo para seguir:

```
    1. Inicializar grafo desde entrada
    2. Calcular flujo máximo sobre el grafo
    3. Para cada nodo de celula calculadora del grafo:
    4.      Calcular flujo maximo sin la celula marcada
    5.      Verificar si es menor a los flujos ya obtenidos (De ser asi, se guarda el flujo y la celula)
    6. Retornar flujo maximo, celula obtenida y flujo excluyendo dicha celula
```



En ese sentido, la complejidad del algoritmo estará fuertemente influenciada por que estrategia se use para resolver el problema de máximo flujo

Esta estrategia, directamente va a explorar que sucede cuando excluimos todos los nodos calculadores. Podria entenderse que es una estrategia de busqueda completa; por lo que explora todas las posibles exclusiones de nodos y busca a partir de esto el nodo especifico que minimiza el paso de flujo

Sin embargo, podemos definir una propiedad que es el flujo que pasa a través de un nodo.

A partir de esto, tendremos en cuenta el siguiente concepto:

* **Flujo del nodo** Se entiende como el flujo que pasa a través del nodo. En ese sentido, este valor se puede hallar sumando todos los flujos que tengan de destino el nodo de interés o sumando todos los flujos que parten a partir del nodo (Por las propiedades de un flujo, sabemos que ambos valores son equivalentes)

Bajo este concepto, se tiene de idea tentativa de que el nodo que se debe excluir para minimizar el flujo total, es aquel cuyo flujo individual sea maximo (Suena enredado, pero se podria probar).

Si esta propiedad es correcta, se tendría el siguiente algoritmo tentativo.


```
    1. Inicializar grafo desde entrada
    2. Calcular flujo máximo sobre el grafo
    3. Identificar el flujo de nodo máximo y su nodo calculador asociado
    4. Recalcular el flujo excluyendo el nodo identificado
    5. Retornar flujo maximo, celula identificada y flujo excluyendo dicha celula
```

In [ ]:
!java ProblemaP2.java < P1.in